In [27]:
! pip install transformers tensorflow transformers[torch] torch torchvision



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd  /content/drive/MyDrive/Data

/content/drive/MyDrive/Data


In [5]:
import zipfile
import os

# Path to the zip file
zip_file_path = '/content/drive/MyDrive/Data/train mango-20230824T105015Z-004.zip'

# Directory to extract the contents
extracted_dir = '/content/drive/MyDrive/Data/extracted_dataset/'

# Create the directory if it doesn't exist
os.makedirs(extracted_dir, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

print('Dataset extracted to:', extracted_dir)

Dataset extracted to: /content/drive/MyDrive/Data/extracted_dataset/


In [31]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from transformers import DeiTFeatureExtractor, DeiTForImageClassification


# Define hyperparameters
batch_size = 16
learning_rate = 0.001
num_epochs = 2
num_classes = len(os.listdir("/content/drive/MyDrive/Data/extracted_dataset/train mango"))

In [32]:
# Data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [33]:
train_dataset = datasets.ImageFolder(root="/content/drive/MyDrive/Data/extracted_dataset/train mango", transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)


### Model loading

In [36]:
# Load DeiT feature extractor and model
model_name = "facebook/deit-base-distilled-patch16-224"
feature_extractor = DeiTFeatureExtractor(model_name)
deit_model = DeiTForImageClassification.from_pretrained(model_name, num_labels=num_classes)


Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(deit_model.parameters(), lr=learning_rate)

### Trainning

In [38]:
# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
deit_model.to(device)

for epoch in range(num_epochs):
    deit_model.train()
    total_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = deit_model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {total_loss:.4f}")

print("Training finished!")

# Save the trained model
torch.save(deit_model.state_dict(), "mango_leaf_classifier.pth")

Epoch [1/2] - Loss: 135.6260
Epoch [2/2] - Loss: 74.5045
Training finished!


### Evaluation

In [39]:
# Load DeiT model and feature extractor
model_name = "facebook/deit-base-distilled-patch16-224"
feature_extractor = DeiTFeatureExtractor(model_name)
deit_model = DeiTForImageClassification.from_pretrained(model_name, num_labels=num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
deit_model.to(device)


# Load the trained model
deit_model.load_state_dict(torch.load("mango_leaf_classifier.pth"))
deit_model.eval()

# Evaluate the model
correct = 0
total = 0

with torch.no_grad():
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = deit_model(images).logits
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Accuracy on the test dataset: {accuracy:.2%}")



Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Accuracy on the test dataset: 78.04%


### interpretations

In [40]:
# Interpretation
class_names = train_dataset.classes

print("Class-wise accuracy:")
class_correct = [0] * num_classes
class_total = [0] * num_classes

with torch.no_grad():
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = deit_model(images).logits
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()

        for i in range(labels.size(0)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(num_classes):
    accuracy = class_correct[i] / class_total[i]
    print(f"Accuracy of {class_names[i]}: {accuracy:.2%}")

Class-wise accuracy:
Accuracy of African Jambo: 97.41%
Accuracy of Bari 8: 99.37%
Accuracy of Indian Totapori: 100.00%
Accuracy of King Brunei: 88.59%
Accuracy of Philippine Honey Dew: 53.68%
Accuracy of Qzai: 91.44%
Accuracy of Taiwani red: 0.00%
Accuracy of baper bari: 45.76%


### Testing

In [46]:
from PIL import Image
# Load and preprocess the test image
def preprocess_image(image_path):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0).to(device)
    return image

# Test the model
test_image_path = "/content/drive/MyDrive/Data/extracted_dataset/18071.png"
image = preprocess_image(test_image_path)
with torch.no_grad():
    outputs = deit_model(image).logits
    predicted_class_idx = outputs.argmax(-1).item()
    predicted_class = class_names[predicted_class_idx]

print(f"Predicted class: {predicted_class}")

Predicted class: King Brunei
